In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
import sys
from pathlib import Path
import anndata as ad
import numpy as np
# import dynamo as dyn
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

sys.path.insert(0, "../../scripts/methods")
from my_SPACEL import spacel_align, spacel_align_ref

sys.path.insert(0, "/home/ylu/project")
from utils import *
from tqdm import tqdm

/home/ylu/anaconda3/envs/SPACEL/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data_folder = "../../data/STARMapPlus_nonrigid_simulation_different_distort/"
results_folder = "./nonrigid_different_distort_results/SPACEL"
Path(results_folder).mkdir(parents=True, exist_ok=True)
cache_folder = os.path.join(results_folder, 'Scube_outputs')
Path(cache_folder).mkdir(parents=True, exist_ok=True)


In [3]:
slice_name = ['A','B','C','D']
tps_slice_name = ['A*','B*','C*','D*']
# i = 3
# data_name = "sagittal{}".format(i)
i = 10
data_name = "well_{}".format(i)
s = ad.read_h5ad('../../data/STARMapPlus/{}.h5ad'.format(data_name))
norm_scale = np.linalg.norm(s.obsm['spatial'].max(0) - s.obsm['spatial'].min(0))
del s
j = 1
slice1 = ad.read_h5ad(os.path.join(data_folder, data_name, "slice_" + str(j) + ".h5ad"))
slice2 = ad.read_h5ad(os.path.join(data_folder, data_name, "slice_" + str(j+1) + ".h5ad"))
repeat = 10

In [ ]:
for l, no in enumerate(np.arange(0,0.22,0.02)):
    if l < 3:
        continue
    alignment_results = []
    for k in range(repeat):
        slice1.obsm['prealign_spatial'] = slice1.obsm['spatial'].copy()
        slice2.obsm['prealign_spatial'] = slice2.obsm['tps_spatial_{}_r_{}'.format(l, k)].copy()
        align_slices, _ = spacel_align_ref(
            models=[slice1.copy(), slice2.copy()],
            spatial_key="prealign_spatial",
            key_added="align_spatial",
            anno_key = 'Main_molecular_tissue_region',
            n_sampling=20000,
        )
        torch.cuda.empty_cache()
        mae, mae_var = MAE(align_slices[1].obsm['align_spatial'], align_slices[1].obsm['spatial']) / norm_scale
        mse, mse_var = MSE(align_slices[1].obsm['align_spatial'], align_slices[1].obsm['spatial']) / norm_scale**2
        print(f'------iteration: {l}----repeat: {k}-----mae: {mae}---------')
        
        alignment_results.append([mae, mae_var, mse, mse_var, data_name, no, l, k])

    np.save(os.path.join(results_folder, data_name + "_SPACEL_differ_distort_{}.npy".format(l)), alignment_results, allow_pickle=True)

Start alignment...
Alignment slice 1 to 0
Runtime: 110.78949856758118 s
------iteration: 3----repeat: 0-----mae: 0.030245327387272224---------
Start alignment...
Alignment slice 1 to 0
Runtime: 111.55250668525696 s
------iteration: 3----repeat: 1-----mae: 0.00829495393617604---------
Start alignment...
Alignment slice 1 to 0


In [ ]:
for l, no in enumerate(np.arange(0,0.22,0.02)):
    alignment_results = []
    for k in range(repeat):
        slice1.obsm['prealign_spatial'] = slice1.obsm['spatial'].copy()
        slice2.obsm['prealign_spatial'] = slice2.obsm['tps_spatial_{}_r_{}'.format(l, k)].copy()
        align_slices, _, _ = paste_align_ref(
        models=[slice1.copy(), slice2.copy()],
            spatial_key="prealign_spatial",
            key_added="align_spatial",
            device="1",
            verbose=False,
            n_sampling=5000,
            sampling_method="random",
        )
        torch.cuda.empty_cache()
        mae, mae_var = MAE(align_slices[1].obsm['align_spatial'], align_slices[1].obsm['spatial']) / norm_scale
        mse, mse_var = MSE(align_slices[1].obsm['align_spatial'], align_slices[1].obsm['spatial']) / norm_scale**2
        
        alignment_results.append([mae, mae_var, mse, mse_var, data_name, no, l, k])

    np.save(os.path.join(results_folder, data_name + "_SPACEL_differ_distort_{}.npy".format(l)), alignment_results, allow_pickle=True)

In [ ]:
# sagittal_morpho_results = {}
spatial_key="prealign_spatial"
key_added="align_spatial"
anno_key = 'Main_molecular_tissue_region'
slice_name = ['A','B','C','D']
tps_slice_name = ['A*','B*','C*','D*']
# for i in range(16):
i = 3
# data_name = "well_{:02d}".format(i)
data_name = "sagittal{}".format(i)
s = ad.read_h5ad('../../data/STARMapPlus/{}.h5ad'.format(data_name))
norm_scale = np.linalg.norm(s.obsm['spatial'].max(0) - s.obsm['spatial'].min(0))
del s

for j in range(3):
    slice1 = ad.read_h5ad(os.path.join(data_folder, data_name, "slice_" + str(j) + ".h5ad"))
    slice2 = ad.read_h5ad(os.path.join(data_folder, data_name, "slice_" + str(j+1) + ".h5ad"))

    for l, no in enumerate(np.arange(0,0.22,0.02)):
        alignment_results = []
        for k in range(repeat):
            slice1.obsm['prealign_spatial'] = slice1.obsm['spatial'].copy()
            slice2.obsm['prealign_spatial'] = slice2.obsm['tps_spatial_{}_r_{}'.format(l, k)].copy()

            sampling_num = 5000
            sampline_idx1 = np.random.choice(slice1.shape[0], sampling_num, replace=False) if slice1.shape[0] > sampling_num else np.arange(slice1.shape[0])
            sampline_idx2 = np.random.choice(slice2.shape[0], sampling_num, replace=False) if slice2.shape[0] > sampling_num else np.arange(slice2.shape[0])
            sub_slice1 = slice1[sampline_idx1,:]
            sub_slice2 = slice2[sampline_idx2,:]
    
            # align slices
            align_slices = spacel_align(
                models=[sub_slice1, sub_slice2],
                spatial_key=spatial_key,
                key_added=key_added,
                anno_key = anno_key,
            )
        
            mae, mae_var = MAE(align_slices[1].obsm['align_spatial'], align_slices[1].obsm['spatial']) / norm_scale
            mse, mse_var = MSE(align_slices[1].obsm['align_spatial'], align_slices[1].obsm['spatial']) / norm_scale**2
    
            alignment_results.append([mae, mae_var, mse, mse_var, data_name, slice_name[j]+tps_slice_name[j+1]])
    
for j in range(3):
    slice1 = ad.read_h5ad(os.path.join(data_folder, data_name, "slice_" + str(j+1) + ".h5ad"))
    slice2 = ad.read_h5ad(os.path.join(data_folder, data_name, "slice_" + str(j) + ".h5ad"))
    slice1.obsm['prealign_spatial'] = slice1.obsm['spatial'].copy()
    slice2.obsm['prealign_spatial'] = slice2.obsm['tps_spatial_0'].copy()

    # align slices
    align_slices = spacel_align(
        models=[slice1, slice2],
        spatial_key=spatial_key,
        key_added=key_added,
        anno_key = anno_key,
    )

    mae, mae_var = MAE(align_slices[1].obsm['align_spatial'], align_slices[1].obsm['spatial']) / norm_scale
    mse, mse_var = MSE(align_slices[1].obsm['align_spatial'], align_slices[1].obsm['spatial']) / norm_scale**2
    
    alignment_results.append([mae, mae_var, mse, mse_var, data_name, slice_name[j+1]+tps_slice_name[j]])

np.save(os.path.join(results_folder, data_name + "_SPACEL.npy"), alignment_results, allow_pickle=True)